***Sử dụng ChormeDriver để crawl dữ liệu title và link của 10-20 bài báo trên đường link:***
https://cafef.vn/tai-chinh-quoc-te.chn

**Lời giải bài tập sử dụng thư viện selenium và beautifulsoup để crawl dữ liệu**

1. Import các thư viện cần dùng

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service 
from bs4 import BeautifulSoup
import pandas as pd 
import time
import chromedriver_autoinstaller
import random

2. Khởi tạo driver

In [19]:
# lấy đường dẫn driver phù hợp với phiên bản chrome hiện tại
driver_path = chromedriver_autoinstaller.install()
# thiết lập các tùy chọn cho driver
option = Options()
# khi lấy dữ liệu thì ẩn việc chrome được lấy dữ liệu sử dụng selenium
option.add_argument("--disable-blink-features=AutomationControlled")
# chạy cửa sổ chrome ở trạng thái toàn màn hình
option.add_argument("start-maximized")
# giả mạo user-agent gửi tới server
option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                    "AppleWebKit/537.36 (KHTML, like Gecko)"
                    "Chrome/124.0.0.0 Safari/537.36")
service = Service(driver_path)

# sử dụng driver với các tham số khởi tạo
driver = webdriver.Chrome(service=service, options=option)
# thời gian đợi trang web phản hồi trước khi báo lỗi
driver.set_page_load_timeout(60)

# url cần crawl
url = 'https://cafef.vn/tai-chinh-quoc-te.chn'

3. Sử dụng beautifulsoup để phân tích mã HTML của trang web

In [20]:
driver.get(url)
# tạo độ ngẫu nhiên giữa các lần truy cập (để không làm quá tải trang web cần crawl)
time.sleep(random.uniform(1, 3))

# lấy mã HTML của toàn bộ web
html = driver.page_source
# khởi tạo đối tượng beautifulsoup để phân tích thông tin mã HTML
soup = BeautifulSoup(html, 'html.parser')
# lấy ra các mục bài báo
contents = soup.find_all("div", class_="tlitem box-category-item")

# tạo list để chứa dữ liệu của từng bài báo
data = []
# đếm số bài báo được crawl (giới hạn 20 bài báo)
cnt = 0

for content in contents:
    if cnt == 20:
        break
    # tiêu đề bài báo
    title = content.h3.a.get_text(strip=True)
    # đường dẫn tới bài báo
    base_link = 'https://cafef.vn'
    link = content.h3.a["href"]
    link = base_link + link
    cnt += 1
    # lưu dưới dạng dictionary
    data.append({
        "Index": cnt,
        "Title": title,
        "Link": link
    })

4. Lưu dữ liệu crawl vào file excel sử dụng thư viện pandas

In [21]:
df = pd.DataFrame(data)
df.to_excel("homework_1.xlsx", index=False)

5. Tắt chrome và tắt phiên làm việc với webdriver


In [22]:
driver.quit()